In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cluster_name = "project1-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # To use a different region for the compute, add a location='<region>' parameter
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({"--C": uniform(0.1, 1.0),"--max_iter": choice(32, 64, 128, 258)})

# Specify a Policy
policy = BanditPolicy(slack_factor=0.2, evaluation_interval=2, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')


# Create a ScriptRunConfig Object to specify the configuration details of your training job
#env = Environment(name="TheresaSampling")
sklearn_env.register(workspace=ws)
src = ScriptRunConfig(source_directory='.',
                        script='train.py',
                        compute_target=cpu_cluster,
                        environment=sklearn_env)


# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                hyperparameter_sampling=ps,
                                policy=policy,
                                max_total_runs=3,
                                primary_metric_name='accuracy',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.widgets import RunDetails
from azureml.train.hyperdrive import HyperDriveRun

hyperdriverun = exp.submit(config=hyperdrive_config)

RunDetails(hyperdriverun).show()

In [ ]:
from azureml.core.model import Model
import os
import joblib
# Get your best run and save the model from that run.
best_run = hyperdriverun.get_best_run_by_primary_metric()

model = best_run.register_model(model_name = 'logistic_regression', model_path = './hyperdrive/logistic_regression.pkl')

model = Model(workspace=ws, name="logistic_regression")
model.download(target_dir=os.getcwd())

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset, Datastore
# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

raw = Dataset.Tabular.from_delimited_files(path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')


In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(raw)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    X=x,
    y=y,
    n_cross_validations=3)

In [2]:
# Submit your automl run

experiment = Experiment(ws, "automlparametersearch")
run = experiment.submit(automl_config, show_output=True)

In [ ]:
# Retrieve and save your best automl model.

import joblib
# Retrieve and save your best automl model.

best_run, best_model = run.get_output()
joblib.dump(value=best_model, filename="automl_logistic_regression.pkl")